In [ ]:
metrics = model.val()
print(metrics.box.map)  # map50-95

In [ ]:
model = YOLO('yolo11m.pt') 

results = model.train(
    data=str(WORKING_DIR / 'bdd100k.yaml'),
    epochs=50,
    imgsz=640,
    batch=16,
    project='bdd100k_yolo_run',
    name='yolo11m_train',
    exist_ok=True,
    pretrained=True,
    optimizer='auto',
    verbose=True,
    seed=42
)

Ultralytics 8.3.234 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/bdd100k.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11m_train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0,

error: OpenCV(4.12.0) :-1: error: (-5:Bad argument) in function 'resize'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'


In [ ]:
yaml_content = {
    'path': str(DATASET_DIR),
    'train': 'images/train',
    'val': 'images/val',
    'names': id_mapping
}

with open(WORKING_DIR / 'bdd100k.yaml', 'w') as f:
    yaml.dump(yaml_content, f, sort_keys=False)

print("Created bdd100k.yaml")
print(yaml.dump(yaml_content, sort_keys=False))

Created bdd100k.yaml
path: /kaggle/working/bdd100k_yolo
train: images/train
val: images/val
names:
  0: person
  1: rider
  2: car
  3: truck
  4: bus
  5: train
  6: motor
  7: bike
  8: traffic light
  9: traffic sign



In [ ]:
def convert_bdd_to_yolo(json_path, images_src_dir, images_dst_dir, labels_dst_dir, img_width=1280, img_height=720):
    """
    Converts BDD100k JSON labels to YOLO format and symlinks images.
    """
    print(f"Processing {json_path}...")
    
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    
    for entry in tqdm(data, desc="Converting labels"):
        filename = entry['name']
        image_src_path = images_src_dir / filename
        
        if not image_src_path.exists():
            continue
            
        image_dst_path = images_dst_dir / filename
        if not image_dst_path.exists():
            os.symlink(image_src_path, image_dst_path)
            
        label_path = labels_dst_dir / (Path(filename).stem + '.txt')
        
        yolo_lines = []
        if 'labels' in entry:
            for label in entry['labels']:
                category = label['category']
                
                if category in class_mapping:
                    class_id = class_mapping[category]
                    
                    if 'box2d' in label:
                        box = label['box2d']
                        x1, y1, x2, y2 = box['x1'], box['y1'], box['x2'], box['y2']
                        
                        dw = 1.0 / img_width
                        dh = 1.0 / img_height
                        
                        w = x2 - x1
                        h = y2 - y1
                        x_center = x1 + w / 2.0
                        y_center = y1 + h / 2.0
                        
                        x_center *= dw
                        w *= dw
                        y_center *= dh
                        h *= dh
                        
                        yolo_lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}")
        
        if yolo_lines:
            with open(label_path, 'w') as f:
                f.write('\n'.join(yolo_lines))

SUBSET = '100k' 

train_json = INPUT_DIR / 'labels' / 'bdd100k_labels_images_train.json'
val_json = INPUT_DIR / 'labels' / 'bdd100k_labels_images_val.json'

train_img_src = INPUT_DIR / 'images' / SUBSET / 'train'
val_img_src = INPUT_DIR / 'images' / SUBSET / 'val'

convert_bdd_to_yolo(train_json, train_img_src, DATASET_DIR / 'images' / 'train', DATASET_DIR / 'labels' / 'train')
convert_bdd_to_yolo(val_json, val_img_src, DATASET_DIR / 'images' / 'val', DATASET_DIR / 'labels' / 'val')

Processing /kaggle/input/bdd-dataset-100k/labels/bdd100k_labels_images_train.json...


Converting labels:   0%|          | 0/69863 [00:00<?, ?it/s]

Processing /kaggle/input/bdd-dataset-100k/labels/bdd100k_labels_images_val.json...


Converting labels:   0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
import cv2
from pathlib import Path
import os
from tqdm.notebook import tqdm

def verify_and_clean_dataset(image_dir, label_dir):
    """
    Verifies images in the dataset and removes corrupt ones (and their labels).
    This prevents OpenCV errors during YOLO training.
    """
    print(f"Verifying images in {image_dir}...")
    image_dir = Path(image_dir)
    label_dir = Path(label_dir)
    
    if not image_dir.exists():
        print(f"Directory not found: {image_dir}")
        return

    image_files = list(image_dir.glob('*.jpg')) + list(image_dir.glob('*.png'))
    
    corrupt_count = 0
    for img_path in tqdm(image_files, desc="Checking images"):
        is_corrupt = False
        try:
            img = cv2.imread(str(img_path))
            
            if img is None:
                print(f"Corrupt/Unreadable image found (None): {img_path}")
                is_corrupt = True
            elif img.size == 0:
                print(f"Corrupt image found (Size 0): {img_path}")
                is_corrupt = True
                
        except Exception as e:
            print(f"Error reading {img_path}: {e}")
            is_corrupt = True
            
        if is_corrupt:
            corrupt_count += 1
            try:
                if img_path.exists() or os.path.islink(img_path):
                    os.remove(img_path)
                    print(f"Deleted image: {img_path}")
                
                label_name = img_path.stem + '.txt'
                label_path = label_dir / label_name
                if label_path.exists():
                    os.remove(label_path)
                    print(f"Deleted label: {label_path}")
            except Exception as del_e:
                print(f"Failed to delete {img_path}: {del_e}")

    print(f"Verification complete. Found and removed {corrupt_count} corrupt images.")

if 'DATASET_DIR' in locals():
    verify_and_clean_dataset(DATASET_DIR / 'images' / 'train', DATASET_DIR / 'labels' / 'train')
    verify_and_clean_dataset(DATASET_DIR / 'images' / 'val', DATASET_DIR / 'labels' / 'val')
else:
    print("DATASET_DIR not defined. Please run the setup cells first.")

In [ ]:
# BDD100k Class Mapping
# We map the BDD100k string labels to integer IDs for YOLO
# BDD Classes: 'bike', 'bus', 'car', 'motor', 'person', 'rider', 'traffic light', 'traffic sign', 'train', 'truck'

class_mapping = {
    'person': 0,
    'rider': 1,
    'car': 2,
    'truck': 3,
    'bus': 4,
    'train': 5,
    'motor': 6,
    'bike': 7,
    'traffic light': 8,
    'traffic sign': 9
}

id_mapping = {v: k for k, v in class_mapping.items()}

print("Class Mapping:", class_mapping)

Class Mapping: {'person': 0, 'rider': 1, 'car': 2, 'truck': 3, 'bus': 4, 'train': 5, 'motor': 6, 'bike': 7, 'traffic light': 8, 'traffic sign': 9}


In [ ]:
import json
import os
import shutil
from pathlib import Path
from tqdm.notebook import tqdm
import yaml
from ultralytics import YOLO

INPUT_DIR = Path('/kaggle/input/bdd-dataset-100k/')
WORKING_DIR = Path('/kaggle/working')
DATASET_DIR = WORKING_DIR / 'bdd100k_yolo'

(DATASET_DIR / 'images' / 'train').mkdir(parents=True, exist_ok=True)
(DATASET_DIR / 'images' / 'val').mkdir(parents=True, exist_ok=True)
(DATASET_DIR / 'labels' / 'train').mkdir(parents=True, exist_ok=True)
(DATASET_DIR / 'labels' / 'val').mkdir(parents=True, exist_ok=True)

print(f"Input Directory: {INPUT_DIR}")
print(f"Dataset Directory: {DATASET_DIR}")

Input Directory: /kaggle/input/bdd-dataset-100k
Dataset Directory: /kaggle/working/bdd100k_yolo


In [ ]:
!pip install -U ultralytics
!pip install -qU "numpy<2.0" "opencv-python-headless==4.10.*"

In [ ]:
!pip uninstall -y opencv-python opencv-python-headless opencv-contrib-python
!pip uninstall -y ultralytics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mkl-fft to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mkl-random to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mkl-umath to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 103.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# BDD100k Object Detection with YOLO11

This notebook demonstrates how to train a YOLO11 object detection model on the BDD100k dataset.
We will:
1.  Install the necessary libraries.
2.  Convert the BDD100k JSON labels to YOLO TXT format.
3.  Prepare the dataset structure using symlinks (to save space and time).
4.  Train a YOLO11x model focusing on vehicles, pedestrians, traffic lights, and signs.

In [ ]:
import glob
import random
import cv2
from IPython.display import Image, display
from PIL import Image as PILImage
import numpy as np

val_images = glob.glob(str(DATASET_DIR / 'images' / 'val' / '*.jpg'))
if len(val_images) > 0:
    random_images = random.sample(val_images, min(3, len(val_images)))

    results = model.predict(random_images)

    for result in results:
        bgr_img = result.plot()
        rgb_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2RGB)
        display(PILImage.fromarray(rgb_img))
else:
    print("No validation images found to visualize.")